In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
from scipy.optimize import differential_evolution
import dill

import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
import numpy as np

from torchvision import datasets
import torch.nn.functional as F
import argparse
from torch.autograd import Variable

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install scipy==1.2.1

In [0]:
CKPT_PATH = '/content/gdrive/My Drive/UofT/CSC421/Project/CSC2516 Project/CheXNet/pneu_model.ckpt'
IMG_PATH_NORM = '/content/gdrive/My Drive/UofT/CSC421/Project/CSC2516 Project/CheXNet/img/normal/'
IMG_PATH_PNEU = '/content/gdrive/My Drive/UofT/CSC421/Project/CSC2516 Project/CheXNet/img/pneumonia/'
ORI_CLASS_NAMES = [ 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', \
					'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia']
BI_ClASS_NAMES = ['Normal', 'Pneumonia']
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
IMG_PATH = "/content/gdrive/My Drive/UofT/CSC421/Project/CSC2516 Project/CheXNet/img_one"

In [0]:
def perturb_image(xs, img):
    if xs.ndim < 2:
        xs = np.array([xs])
    batch = len(xs)
    imgs = img.repeat(batch, 1, 1, 1)
    xs = xs.astype(int)

    count = 0
    for x in xs:
        pixels = np.split(x, len(x) / 5)

        for pixel in pixels:
            x_pos, y_pos, r, g, b = pixel
            imgs[count, 0, x_pos, y_pos] = (r / 255.0 - 0.4914) / 0.2023
            imgs[count, 1, x_pos, y_pos] = (g / 255.0 - 0.4822) / 0.1994
            imgs[count, 2, x_pos, y_pos] = (b / 255.0 - 0.4465) / 0.2010
        count += 1

    return imgs

In [0]:
def predict_classes(xs, img, target_calss, net, minimize=True):
    imgs_perturbed = perturb_image(xs, img.clone())
    input = Variable(imgs_perturbed).to(device)
    predictions = F.softmax(net(input)).data.cpu().numpy()[:, target_calss]

    return predictions if minimize else 1 - predictions

In [0]:
def attack_success(x, img, target_class, net, targeted_attack=False, verbose=False):
    attack_image = perturb_image(x, img.clone())
    input = Variable(attack_image).to(device)
    confidence = F.softmax(net(input)).data.cpu().numpy()[0]
    predicted_class = np.argmax(confidence)

    if (verbose):
        print("Confidence: %.4f" % confidence[target_class])
    if (targeted_attack and predicted_class == target_class) or (
            not targeted_attack and predicted_class != target_class):
        return True

In [0]:
def attack(img, label, net, target=None, pixels=1, maxiter=75, popsize=400, verbose=False):
    # img: 1*3*W*H tensor
    # label: a number

    targeted_attack = target is not None
    target_class = target if targeted_attack else label

    bounds = [(0, 224), (0, 224), (0, 255), (0, 255), (0, 255)] * pixels

    popmul = max(1, popsize / len(bounds))

    predict_fn = lambda xs: predict_classes(
        xs, img, target_class, net, target is None)
    callback_fn = lambda x, convergence: attack_success(
        x, img, target_class, net, targeted_attack, verbose)

    inits = np.zeros([int(popmul * len(bounds)), len(bounds)])
    print(inits.shape)
    for init in inits:
        for i in range(pixels):
            init[i * 5 + 0] = np.random.random() * 224
            init[i * 5 + 1] = np.random.random() * 224
            init[i * 5 + 2] = np.random.normal(128, 127)
            init[i * 5 + 3] = np.random.normal(128, 127)
            init[i * 5 + 4] = np.random.normal(128, 127)

    attack_result = differential_evolution(predict_fn, bounds, maxiter=maxiter, popsize=popmul, mutation=(0.5, 1),
                                           recombination=0.7, atol=-1, callback=callback_fn, polish=True, init=inits)

    attack_image = perturb_image(attack_result.x, img)
    attack_var = Variable(attack_image).to(device)
    predicted_probs = F.softmax(net(attack_var)).data.cpu().numpy()[0]

    predicted_class = np.argmax(predicted_probs)

    if (not targeted_attack and predicted_class != label) or (targeted_attack and predicted_class == target_class):
        return 1, attack_result.x.astype(int)
    return 0, [None]

In [0]:
def attack_all(net, loader, pixels=1, targeted=False, maxiter=75, popsize=400, verbose=False):
    correct = 0
    success = 0

    for batch_idx, (input, target) in enumerate(loader):

        img_var = Variable(input).to(device)
        torch.no_grad()

        prior_probs = F.softmax(net(img_var), dim=1)
        _, indices = torch.max(prior_probs, 1)

        if target[0] != indices.data.cpu()[0]:
            continue

        correct += 1
        target = target.numpy()

        targets = [None] if not targeted else range(2)

        for target_calss in targets:
            if (targeted):
                if (target_calss == target[0]):
                    continue

            flag, x = attack(input, target[0], net, target_calss, pixels=pixels, maxiter=maxiter, popsize=popsize,
                             verbose=verbose)

            success += flag
            if (targeted):
                success_rate = float(success) / (1 * correct)
            else:
                success_rate = float(success) / correct

            if flag == 1:
                print("success rate: %.4f (%d/%d) [(x,y) = (%d,%d) and (R,G,B)=(%d,%d,%d)]" % (success_rate, success, correct, x[0], x[1], x[2], x[3], x[4]))

        if correct == args.samples:
            break

    return success_rate

In [0]:
def main():
    print("==> Loading data and model...")


    CheXnet_model = loadChexNetModel(CKPT_PATH)

    fileFolder = IMG_PATH


    data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

    image_dataset = datasets.ImageFolder(fileFolder, data_transform)
    dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=1, shuffle=False, num_workers=0)

    print("==> Starting attck...")

    results = attack_all(CheXnet_model, dataloader, pixels=args.pixels, targeted=args.targeted, maxiter=args.maxiter,
                         popsize=args.popsize, verbose=args.verbose)
    print("Final success rate: %.4f" % results)

In [0]:
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

In [0]:
def loadChexNetModel(model_path):
	out_size = 2
	model = torchvision.models.densenet121(pretrained=True)
	num_ftrs = model.classifier.in_features
	model.classifier = nn.Sequential(
		nn.Linear(num_ftrs, out_size),
		nn.Softmax(1)
	)

	# load checkpoint
	if torch.cuda.is_available():
	    state_dict = torch.load(model_path)
	else:
	    state_dict = torch.load(model_path, map_location='cpu')

	model.load_state_dict(state_dict)
	return model.eval().to(device)

In [0]:
parser = argparse.ArgumentParser(description='One pixel attack with PyTorch')
parser.add_argument('--pixels', default=28, type=int, help='The number of pixels that can be perturbed.')
# parser.add_argument('--maxiter', default=100, type=int, help='The maximum number of iteration in the DE algorithm.')
parser.add_argument('--maxiter', default=100, type=int, help='The maximum number of iteration in the DE algorithm.')
parser.add_argument('--popsize', default=400, type=int, help='The number of adverisal examples in each iteration.')
# parser.add_argument('--popsize', default=16, type=int, help='The number of adverisal examples in each iteration.')
parser.add_argument('--samples', default=100, type=int, help='The number of image samples to attack.')
# parser.add_argument('--samples', default=4, type=int, help='The number of image samples to attack.')
parser.add_argument('--targeted', action='store_true', help='Set this switch to test for targeted attacks.')
parser.add_argument('--verbose', action='store_true', help='Print out additional information every iteration.')

#args = parser.parse_args("--verbose --targeted")

_StoreTrueAction(option_strings=['--verbose'], dest='verbose', nargs=0, const=True, default=False, type=None, choices=None, help='Print out additional information every iteration.', metavar=None)

In [0]:
args = parser.parse_args("--verbose".split())

In [0]:
main()

==> Loading data and model...
==> Starting attck...
(400, 140)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  after removing the cwd from sys.path.


Confidence: 0.6771
Confidence: 0.6615
Confidence: 0.6224
Confidence: 0.6224
Confidence: 0.6224
Confidence: 0.6224
Confidence: 0.6224
Confidence: 0.6224
Confidence: 0.6224
Confidence: 0.6224
Confidence: 0.6224
Confidence: 0.6224
Confidence: 0.6224
Confidence: 0.6224
Confidence: 0.6065
Confidence: 0.6065
Confidence: 0.6065
Confidence: 0.6065
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5967
Confidence: 0.5199
Confidence: 0.5199
Confidence: 0.5199
Confidence: 0.5199
Confidence: 0.5199
Confidence: 0.5199
Confidence: 0.5199
Confidence: 0.5199
Confidence: 0.5199
Confidence: 0.5199
Confidence: 0.5199
Confidence: 0.5199
Confidence: 0.5199
Confidence: 0.5199
Confidence: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


success rate: 1.0000 (1/1) [(x,y) = (139,57) and (R,G,B)=(205,139,242)]
(400, 140)
Confidence: 0.7258
Confidence: 0.7258
Confidence: 0.7258
Confidence: 0.7258
Confidence: 0.7258
Confidence: 0.7258
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7257
Confidence: 0.7255
Confidence: 0.7255
Confidence: 0.7255
Confi

In [0]:
|